In [9]:
include("headers.jl")
using PyPlot, Statistics

In [10]:
q = 2
mult, gfinv = gftables(q);   # Matrices for multiplication and inverse on GF
algo = MS();

## Max-Sum with convergence criterion: decisional variables

In [11]:
n = 480
nedges = n*2
niter = 50
maxiter=Int(5e3)
L = 1
converged = 0
parity = 0
lambda = [0.0, 1.0]
rho = [0.0, 0.0, 0.5, 0.5]
k = Int(round(nedges*(sum(rho[j]/j for j in eachindex(rho)))))
R = k/n

0.5833333333333334

In [9]:
FG = ldpc_graph(q, n, k, nedges, lambda, rho, verbose=false)
for it in 1:niter
    y = rand(0:q-1, n)
    FG.fields .= extfields(q,y,L)
    res = bp!(FG,algo, verbose=false, maxiter=maxiter, nmin=500)
    if res == :unconverged
#         println("Unconverged with n=$n, k=$(k) after $maxiter steps")
#         println()
    else
        converged += 1
        z = paritycheck(FG, algo)
        if sum(z)!=0 
#             println("Parity not fulfilled: $(sum(z)) unfulfilled constraints")
        else
#             println("PARITY FULFILLED")
            parity += 1
        end
#         println()
    end
    mod(it,5)==0 && println("Finished iter $it")
end
println("Converged $converged out of $niter times.")
println("Parity check fulfilled $parity out of $converged times.")

Finished iter 5
Finished iter 10
Finished iter 15
Finished iter 20
Finished iter 25
Finished iter 30
Finished iter 35
Finished iter 40
Finished iter 45
Finished iter 50
Converged 14 out of 50 times.
Parity check fulfilled 14 out of 14 times.


### Varying rate

In [37]:
npoints = 4
navg = 50
n = 840
nedges = n*2
maxiter=Int(5e3)
L = 1
Rho = [zeros(npoints+4) for _ in 1:npoints]
K = zeros(Int,npoints)
for x in 1:npoints
    Rho[x][x+3:x+4] = [0.5 0.5]
    K[x] = Int(round(n*(1/(x+3)+1/(x+4))))
end
R = K./n
dist = zeros(npoints, navg)
converged = zeros(Int, npoints)
parity = zeros(Int, npoints);

In [38]:
for j in 1:npoints
    FG = ldpc_graph(q, n, K[j], nedges, lambda, Rho[j], verbose=false)
    for it in 1:navg
        y = rand(0:q-1, n)
        FG.fields .= extfields(q,y,L)
        res = bp!(FG,algo, verbose=false, maxiter=maxiter, nmin=500)
        if res != :unconverged
            converged[j] += 1
        end
        sum(paritycheck(FG, algo))==0 && (parity[j] += 1)
        dist(j,it) = hd(res,y)/n
        mod(it,10)==0 && println("Finished iter $it")
    end
    println("---Finished R=$(R[j])---")
end

Finished iter 10
Finished iter 20
Finished iter 30
Finished iter 40
Finished iter 50
---Finished R=0.45---
Finished iter 10
Finished iter 20
Finished iter 30
Finished iter 40
Finished iter 50
---Finished R=0.36666666666666664---
Finished iter 10
Finished iter 20
Finished iter 30
Finished iter 40
Finished iter 50
---Finished R=0.30952380952380953---
Finished iter 10
Finished iter 20
Finished iter 30
Finished iter 40
Finished iter 50
---Finished R=0.26785714285714285---


In [41]:
println("n=$n")
println()
for j in 1:npoints
   println("R=$(R[j])")
    println("Converged $(converged[j]) out of $navg times.")
    println("Parity check fulfilled $(parity[j]) out of $navg times.")
    println()
end

n=840

R=0.45
Converged 16 out of 50 times.
Parity check fulfilled 10 out of 50 times.

R=0.36666666666666664
Converged 18 out of 50 times.
Parity check fulfilled 15 out of 50 times.

R=0.30952380952380953
Converged 18 out of 50 times.
Parity check fulfilled 9 out of 50 times.

R=0.26785714285714285
Converged 22 out of 50 times.
Parity check fulfilled 2 out of 50 times.



In [45]:
sum(dist)

0.0

## Max-Sum with convergence criterion: messages

In [12]:
n = 480
nedges = n*2
niter = 3
maxiter=Int(5e3)
L = 1
lambda = [0.0, 1.0]
rho = [0.0, 0.0, 0.5, 0.5]
nedges = 2*n
k = Int(round(nedges*(sum(rho[j]/j for j in eachindex(rho)))))
R = k/n

0.5833333333333334

In [ ]:
converged = 0
parity = 0

FG = ldpc_graph(q, n, k, nedges, lambda, rho, verbose=false)
for it in 1:niter
    y = rand(0:q-1, n)
    FG.fields .= extfields(q,y,L)
    res = bp_msg!(FG,algo, verbose=false, maxiter=maxiter, tol=1e-6)
    if res == :unconverged
    else
        converged += 1
        z = paritycheck(FG, algo)
        if sum(z)==0 
            parity += 1
        end
    end
    mod(it,10)==0 && println("Finished iter $it")
end
println("Converged $converged out of $niter times.")
println("Parity check fulfilled $parity out of $converged times.")